
# 06 — Programmation Dynamique (DP) — Version **Université**

> **Objectif pédagogique** : savoir **reconnaître** un problème de Programmation Dynamique, **construire** la bonne formulation (`dp[...]`), **choisir** entre mémoïsation (top‑down) et tabulation (bottom‑up), **reconstruire** une solution optimale, et **expliquer** les complexités en entretien.
>
> **Applications trading/data** : allocation sous contrainte (risk budget), rebalancing discret, édition de flux (distance d’édition), filtrage de séquences, matching d’événements.

## Plan du cours
1. Motivation & intuition de la DP  
2. Les deux questions pour **détecter** la DP  
3. Mémoïsation (top‑down) vs Tabulation (bottom‑up)  
4. Méthode systématique pour modéliser un `dp`  
5. Classiques : **Knapsack 0/1**, **Coin Change**, **Edit Distance**, **LIS** (longest increasing subsequence)  
6. Reconstruction des solutions  
7. Pièges, bonnes pratiques, checklist d’entretien  
8. Exercices guidés + corrigés



## 1) Motivation & intuition

La **DP** sert quand :
- on peut **décomposer** un problème en **sous‑problèmes qui se répètent**,
- l’**optimal global** se construit à partir d’**optimaux locaux** (structure optimale).

Exemples concrets :
- **Knapsack** : sélectionner des positions sous une contrainte de **budget/risque** pour maximiser un score.
- **Coin Change** : effectuer un **arrondi discret** au plus juste (ticks, tailles de lots).
- **Edit Distance** : mesurer la **différence** entre deux séquences (flux d’ordres vs exécutions).
- **LIS** : extraire une **tendance croissante** la plus longue (filtrage de bruit sur séries discrètes).



## 2) Détecter la DP : 2 questions simples

1. **Sous‑problèmes qui se recoupent ?**  
   Les mêmes calculs reviennent à différents endroits (ex. même suffixe/préfixe).

2. **Structure optimale ?**  
   L’optimal global peut‑il être construit en combinant des **choix optimaux** sur des sous‑espaces ?

Si **oui** → grande chance que la DP soit la bonne approche.



## 3) Mémoïsation vs Tabulation

- **Mémoïsation (top‑down)** : on écrit une **récursion** naïve puis on **mémorise** les résultats. Très rapide à prototyper avec `@lru_cache`.
- **Tabulation (bottom‑up)** : on **remplit** une table selon un **ordre de calcul**. Plus contrôlable, souvent plus **mémoire‑économe** et **prévisible**.


In [1]:

from functools import lru_cache

@lru_cache(maxsize=None)
def fib_memo(n: int) -> int:
    if n < 2: return n
    return fib_memo(n-1) + fib_memo(n-2)

def fib_tabu(n: int) -> int:
    if n < 2: return n
    a, b = 0, 1
    for _ in range(2, n+1):
        a, b = b, a + b
    return b

[fib_memo(i) for i in range(10)], [fib_tabu(i) for i in range(10)]


([0, 1, 1, 2, 3, 5, 8, 13, 21, 34], [0, 1, 1, 2, 3, 5, 8, 13, 21, 34])


## 4) Méthode systématique pour modéliser un `dp`

1. **État** : que représente `dp[i][...]` ? (ex. meilleure valeur en considérant les `i` premiers éléments)  
2. **Transition** : comment calculer `dp[...]` depuis des états plus simples ? (formule de récurrence)  
3. **Bords** : quelles sont les conditions initiales ?  
4. **Ordre de calcul** : dans quel ordre remplir la table ?  
5. **Réponse** : où se trouve la solution dans la table ?  
6. **Reconstruction** (souvent oublié) : comment retrouver **les choix** menant à l’optimal ?



## 5) Knapsack 0/1 — valeur **et reconstruction** (indispensable en entretien)

**Énoncé** : on a des poids `w[i]`, valeurs `v[i]`, capacité `W`. On veut **maximiser** la valeur sans **dépasser** `W`.

**Modélisation**
- **État** : `dp[i][w]` = meilleure valeur en utilisant les `i` premiers objets avec capacité `w`.
- **Transition** :
\-\- si on **ne prend pas** l’objet `i` : `dp[i-1][w]`  
\-\- si on **prend** l’objet `i` (si `w_i ≤ w`) : `dp[i-1][w - w_i] + v_i`  
⇒ `dp[i][w] = max(dp[i-1][w], dp[i-1][w-w_i] + v_i)`

- **Bords** : `dp[0][w] = 0` pour tout `w`.
- **Réponse** : `dp[n][W]`.
- **Reconstruction** : on remonte la table pour lister les objets choisis.


In [2]:

def knapsack_reconstruct(weights, values, W):
    n = len(weights)
    dp = [[0]*(W+1) for _ in range(n+1)]
    for i in range(1, n+1):
        wi, vi = weights[i-1], values[i-1]
        for w in range(W+1):
            dp[i][w] = dp[i-1][w]
            if wi <= w:
                cand = dp[i-1][w-wi] + vi
                if cand > dp[i][w]:
                    dp[i][w] = cand
    # Reconstruction des choix
    w = W; take = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:
            take.append(i-1)
            w -= weights[i-1]
    take.reverse()
    return dp[n][W], take

val, items = knapsack_reconstruct([2,3,4,5], [3,4,5,8], W=8)
val, items  # valeur optimale et indices pris


(12, [1, 3])


## 6) Coin Change — **minimum** de pièces

**Énoncé** : rendre un montant `amount` avec un set de pièces `coins`; minimiser le **nombre de pièces**.

**Modélisation**
- **État** : `dp[a]` = nombre minimal de pièces pour atteindre `a`.
- **Transition** : `dp[a] = min_{c∈coins}( dp[a-c] + 1 )` si `c ≤ a`.
- **Bords** : `dp[0]=0`, et infini pour le reste avant calcul.
- **Réponse** : `dp[amount]` (ou `-1` si impossible).


In [3]:

def coin_change_min(coins, amount):
    INF = 10**9
    dp = [0] + [INF]*amount
    for a in range(1, amount+1):
        for c in coins:
            if c <= a:
                dp[a] = min(dp[a], dp[a-c] + 1)
    return dp[amount] if dp[amount] < INF else -1

coin_change_min([1,2,5], 11)  # 3 (5+5+1)


3


## 7) Edit Distance (Levenshtein) — tabulation complète

**Énoncé** : transformer `a` en `b` via insertions/suppressions/substitutions (coût 1).  
**Modélisation**
- **État** : `dp[i][j]` = coût minimal pour `a[:i]` → `b[:j]`.
- **Transitions** :
  - Suppression : `dp[i-1][j] + 1`
  - Insertion  : `dp[i][j-1] + 1`
  - Substitution (ou égalité) : `dp[i-1][j-1] + cost` (`0` si `a[i-1]==b[j-1]`, sinon `1`)
- **Bords** : `dp[i][0]=i`, `dp[0][j]=j`.
- **Complexité** : \(O(nm)\).


In [4]:

def edit_distance(a: str, b: str) -> int:
    n, m = len(a), len(b)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(n+1): dp[i][0] = i
    for j in range(m+1): dp[0][j] = j
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,      # suppression
                dp[i][j-1] + 1,      # insertion
                dp[i-1][j-1] + cost  # substitution/égalité
            )
    return dp[n][m]

edit_distance("kitten", "sitting")


3


## 8) LIS — Longest Increasing Subsequence (O(n log n))

**Idée** : maintenir `tails[k]` = **plus petite fin possible** d’une sous‑séquence croissante de longueur `k+1`.  
Pour chaque `x`, on trouve par **recherche binaire** la première position `i` où `tails[i] ≥ x`, et on remplace.

- **Complexité** : \(O(n \log n)\) pour la **longueur** seulement.
- **Reconstruction** : nécessite de garder les **indices précédents**.


In [5]:

from bisect import bisect_left

def lis_length(a):
    tails = []
    for x in a:
        i = bisect_left(tails, x)
        if i == len(tails):
            tails.append(x)
        else:
            tails[i] = x
    return len(tails)

lis_length([10,9,2,5,3,7,101,18])


4


### 8.1 Reconstruction de la séquence


In [6]:

def lis_reconstruct(a):
    from bisect import bisect_left
    tails = []
    tails_idx = []
    prev = [-1]*len(a)
    for i, x in enumerate(a):
        j = bisect_left(tails, x)
        if j == len(tails):
            tails.append(x); tails_idx.append(i)
        else:
            tails[j] = x; tails_idx[j] = i
        if j > 0:
            prev[i] = tails_idx[j-1]
    # Reconstruire depuis le dernier index
    res = []
    k = tails_idx[-1] if tails_idx else -1
    while k != -1:
        res.append(a[k]); k = prev[k]
    return res[::-1]

lis_reconstruct([10,9,2,5,3,7,101,18])


[2, 3, 7, 18]


## 9) Pièges, bonnes pratiques, checklist d’entretien

**Pièges courants**
- Mauvaise **définition d’état** (`dp[...]` ne capture pas la “mémoire” nécessaire).
- Oublier les **conditions de bord** (lignes/colonnes 0).
- Remplir la table dans un **mauvais ordre** (utilisation de valeurs non encore calculées).
- **Oublier la reconstruction** (souvent demandée en entretien).

**Bonnes pratiques**
- Écrire **d’abord** en mots l’état et la transition.
- Commencer par **tabulation** pour clarifier l’ordre de calcul, puis optimiser (1D, rolling arrays).
- **Valider** sur des petits cas où la réponse est connue.

**Checklist entretien**
- Expliquer **état/transition/bords/ordre/réponse** en 30 secondes.  
- Donner la **complexité** en temps et en mémoire.  
- Montrer la **reconstruction** pour au moins un problème.



## 10) Exercices guidés

### Exercice A — Knapsack 1D (optimisation mémoire)
**Tâche** : transformer le knapsack 2D en **table 1D** (`dp[w]`) en itérant `w` **à l’envers**.  
**Indice** : pour ne pas réutiliser un item plusieurs fois, on parcourt `w` de `W` à `wi`.

> 👉 Corrigé ci‑dessous.


In [7]:

def knapsack_1d(weights, values, W):
    dp = [0]*(W+1)
    for wi, vi in zip(weights, values):
        for w in range(W, wi-1, -1):   # ← ordre inverse crucial
            dp[w] = max(dp[w], dp[w-wi] + vi)
    return dp[W]

knapsack_1d([2,3,4,5], [3,4,5,8], 8)


12


### Exercice B — Edit Distance : **reconstruction** (optionnel)
**Tâche** : modifier `edit_distance` pour **reconstruire** une suite d’opérations (I, D, S) menant de `a` à `b`.

*(À faire en autonomie — indice : garder une table “parent” pour remonter les choix.)*
